# Efficiently representing radio astronomy data in qbits

## Creating Antenna Fields

In [173]:
import numpy as np

In [174]:
class Antenna:
    #half-square complex values / other half conjugates
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def get_complex_value(self):
        return complex(self.x, self.y)
        
    def __repr__(self):
        return str(complex(self.x, self.y))
  

In [175]:
antennas = []
n = 2
for i in range(0, n):
    antennas.append(Antenna(np.random.rand(), np.random.rand()))
                    
print(antennas)

[(0.4060549702881697+0.25142779511850144j), (0.7641309880413082+0.1611412362519723j)]


In [176]:
visibilities = np.zeros((n, n , 4)) #only looking at antennas inner loops
XX = 0
XY = 1
YX = 2
YY = 3

for i in range(0, len(antennas)):
    for j in range(0, i+1):
        visibilities[i][j][XX] = antennas[i].x * antennas[j].x
        #visibilities[j][i][XX] = np.conj(visibilities[i][j][XX])
        visibilities[i][j][XY] = antennas[i].x * antennas[j].y
        #visibilities[j][i][XY] = np.conj(visibilities[i][j][XY])
        visibilities[i][j][YX] = antennas[i].y * antennas[j].x
        #visibilities[j][i][YX] = np.conj(visibilities[i][j][YX])
        visibilities[i][j][YY] = antennas[i].y * antennas[j].y
        #visibilities[j][i][YY] = np.conj(visibilities[i][j][YY])

print(visibilities)

[[[0.16488064 0.10209351 0.10209351 0.06321594]
  [0.         0.         0.         0.        ]]

 [[0.31027919 0.19212377 0.0654322  0.04051539]
  [0.58389617 0.12313301 0.12313301 0.0259665 ]]]


In [177]:
zeros_amount = ((visibilities.shape[0] * visibilities.shape[1] * visibilities.shape[2]) - (visibilities.shape[0] * visibilities.shape[2])) / 2
nonzero_values = int((visibilities.shape[0] * visibilities.shape[1] * visibilities.shape[2]) - zeros_amount)
number_of_bits = 32
off_set = 0
qc = QuantumCircuit(nonzero_values*number_of_bits)
for i in range(0, visibilities.shape[0]):
    for j in range(0, i+1):
        for k in range(0, visibilities.shape[2]):
            binary = float_to_bin(visibilities[i, j, k])
            off_set = encoding2(qc, binary, off_set)
            
qc.measure_all()
#qc.draw('mpl')

backend = Aer.get_backend('aer_simulator')
job = backend.run(qc, shots=1, memory=True)
output = job.result().get_memory()[0]
out = reverse(output)

chunks = []
for i in range(0, nonzero_values):
    chunks.append(out[number_of_bits*i:number_of_bits+(number_of_bits*i)])
readout = []
for i in range(0, len(chunks)):
    readout.append(bin_to_float(chunks[i]))
print(readout)    

[0.164880633354187, 0.10209350287914276, 0.10209350287914276, 0.06321593374013901, 0.3102791905403137, 0.19212377071380615, 0.06543219834566116, 0.04051538556814194, 0.5838961601257324, 0.12313301116228104, 0.12313301116228104, 0.025966497138142586]


## Naive representations

In [178]:
import qiskit
from qiskit import *
from qiskit.visualization import plot_histogram
import numpy as np

### Circuit family #1

In [179]:
def encoding2(qc, binary, off_set):
    
    for i in range(0, len(binary)):
        qc.reset(off_set+i)

        if binary[i]=='1':
            qc.x(off_set+i)
    
    off_set += len(binary)
    
    """
    qc.barrier()
    qc.measure_all()
    qc.draw('mpl')

    backend = Aer.get_backend('aer_simulator')
    job = backend.run(qc, shots=1, memory=True)
    output = job.result().get_memory()[0]
    """
    return off_set

## Utils

In [181]:
#float to binary / binary to float
import struct

def float_to_bin(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')

def bin_to_float(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]

In [182]:
def reverse(string):
    string = string[::-1]
    return string